In [20]:
import json
import pandas as pd
from collections import Counter, defaultdict
import os

In [21]:
os.listdir('data/z')

['3.json', '0.json', '1.json', '2.json', '4.json']

In [2]:
df = pd.read_csv('Ad Authenticity DB - Sheet1.csv')
df.head()a

,https://www.alexa.com/topsites/category/News,Unnamed: 1,Unnamed: 2
0,Publisher URL (Home page),Story Page,JSON LD
1,https://www.reddit.com/,https://www.reddit.com/r/politics/comments/bn1...,None
2,https://www.washingtonpost.com,https://www.washingtonpost.com/politics/trump-...,"\n{\n ""@context"":""http://schema.org"",\n\n \n..."
3,https://www.washingtonpost.com,https://www.washingtonpost.com/business/2019/0...,"\n{\n ""@context"":""http://schema.org"",\n\n \n..."
4,NaN,NaN,NaN


In [3]:
schemas = df['Unnamed: 2'][1:]
schemas

1                                                  None
2     \n{\n  "@context":"http://schema.org",\n\n  \n...
3     \n{\n  "@context":"http://schema.org",\n\n  \n...
4                                                   NaN
5     \n{\n  "@context":"http://schema.org",\n\n  \n...
6                                                  None
7     \n      {\n        "name": "Los Angeles Times"...
8     \n      {\n        "@context": "http://schema....
9     \n{\n    "@context": "http://schema.org",\n   ...
10                                                 None
11    \n {\n   "@context": "http://schema.org",\n   ...
12                                                 None
13    \n        {\n            "@context": "http://s...
14                                                 None
15                                                 None
16                                                 None
17                                                 None
18                                              

In [4]:
data = []
for item in schemas:
    try:
        data.append(json.loads(str(item)))
    except json.JSONDecodeError as e:
        pass
len(data)

24

In [5]:
with open('compiled.json', 'w') as f:
    f.write(json.dumps(data))

In [13]:
publisher_types = []
content_types =  []
publisher_names = []
authors = []
keywords = []
for d in data:
    pub = d.get('publisher', {'@type': None, 'name': None})
    publisher_types.append(pub['@type'])
    publisher_names.append(pub['name'])
    content_types.append(d['@type'])
    authors.append(d.get('author', {}))
    keywords.append(d.get('keywords', []))

In [19]:
keyword_by_publisher = defaultdict(Counter)
for d in data:
    pub = d.get('publisher', {'@type': None, 'name': None})
    keywords = d.get('keywords', [])
    if isinstance(keywords, str):
        keywords = keywords.split(',')
    keyword_by_publisher[pub['name']].update(keywords)
keyword_by_publisher

defaultdict(collections.Counter,
            {'The Washington Post': Counter(),
             None: Counter(),
             'Los Angeles Times': Counter({'Uber': 1,
                      'China': 1,
                      'Snap Inc.': 1,
                      'Garrett Camp': 1,
                      'Alibaba Group': 1}),
             'oregonlive.com': Counter({'@breaking-news': 6,
                      '@hp-top-stories': 6,
                      '@oregon-breaking': 1}),
             'Wall Street Journal': Counter({'audit': 1,
                      'congressional investigations': 1,
                      'democrats': 1,
                      'irs': 1,
                      'probes': 1,
                      'subpoena': 1,
                      'tax refund': 1,
                      'tax returns': 1,
                      'trump taxes': 1,
                      'where is my refund': 1,
                      'Rep. Richard Neal': 1,
                      'Trump': 1,
                      'Ri

In [8]:
Counter(publisher_types)

Counter({'NewsMediaOrganization': 5, None: 6, 'Organization': 13})

In [9]:
Counter(publisher_names)

Counter({'The Washington Post': 3,
         None: 6,
         'Los Angeles Times': 1,
         'oregonlive.com': 6,
         'Wall Street Journal': 1,
         'Libération': 1,
         'Times Of India': 1,
         'CNBC': 1,
         'MSNBC': 1,
         'Houston Chronicle': 1,
         'San Francisco Chronicle': 1,
         'Hindustan Times': 1})

In [10]:
Counter(content_types)

Counter({'ReportageNewsArticle': 3,
         'Organization': 3,
         'NewsArticle': 15,
         'BreadcrumbList': 3})

In [11]:
author_names = []

for x in authors:
    try:
        author_names.append(x[0]['name'])
    except KeyError as e:
        pass
Counter(author_names)

Counter({'Suhauna Hussain ': 1,
         'Noelle Crombie | The Oregonian/OregonLive': 1,
         'Alain Auffray': 1,
         'Julian Gill': 1,
         'Amy Graff': 1,
         'Gordon R. Friedman | The Oregonian/OregonLive': 5})